# Simulation 

### Import des librairies

In [46]:
import numpy as np
import random 
import simpy
import time
import csv
from collections import namedtuple
from datetime import datetime
from datetime import timedelta

### Fonctions internes à la simulation

In [47]:
def imps():
    """Fonction qui génère un nombre d'impressions devant un panneau"""
    lam = int(np.random.normal(loc=4, scale=2, size=1))
    if lam < 1:
        lam = 1
    nb_imp = np.random.poisson(lam)
    return nb_imp

In [48]:
def delai(lam):
    """Fonction qui génère un nombre de secondes avant la prochaine br grâce à une loi de poisson de paramètre lambda"""
    secondes = np.random.poisson(lam)
        
    #Simuler une proba de pb technique (1% de chance d'en avoir un)
    if not random.random() < 0.99:
        secondes = np.random.poisson(secondes + 1000)
    return secondes

In [49]:
def sauvegarde(liste, nom_de_fichier):
    """Fonction pour sauvegarder un named tuple dans un format CSV"""
    with open(nom_de_fichier, "w", encoding="utf8") as fichier:
        #Recuperer le nom des colonnes pour la première ligne
        premier, *_ = liste
        ecrivain = csv.DictWriter(fichier, premier._fields)
        ecrivain.writeheader()
        for br in liste:
            # On ecrit chaque ligne comme un ordered dict
            ecrivain.writerow(br._asdict())

### Fonction de la simulation

In [50]:
def open_rtb_pacing_double_algo(env, P, timestampnow, nb_jours, nb_hours_per_day, bidrequests, data):
    """Fonction pour générer la simulation"""
    day = datetime.now().day
    month = datetime.now().month
    year = datetime.now().year
    next_date = datetime(year,month,day,6,0,0,0)
    setup = True
    identifiant = 0
    while True:
        # Déterminer dans quel état on est actuellement
        current_hour = datetime.fromtimestamp(env.now).hour
        if current_hour >= 6 and current_hour < 20:
            if setup:
                setup = False
                next_date += timedelta(days=1)
                lambdas_seuils = np.array([1,60,30,5])
                timestamps_seuils = np.array([env.now,env.now+32400,env.now+39600,env.now+54000])
                f_lambdas = interp1d(timestamps_seuils, lambdas_seuils)
                
            # Génération d'une br
            identifiant += 1
            
            #Timestamp de la br
            time = datetime.fromtimestamp(env.now).strftime("%m-%d-%Y %H:%M:%S")

            # Nombre d'impressions
            nb_imp = imps()
            Prix = P * nb_imp

            # Stocker les résultats dans le namedtuple puis ajout à une liste
            resultats = bidrequests(
                identifiant = identifiant,
                timestamp = env.now,
                timestamp_string = time,
                nombre_impressions = nb_imp,
                prix_impression = P,
                prix_total = Prix
            )
            data.append(resultats)

            # Temps avant la prochaine BR
            time_before_next = delai(f_lambdas(env.now))
            
            # Temps restant avant la fin de la simulation
            rt = timestampnow + timedelta(days = nb_jours).total_seconds() - env.now

            # Fin de la simulation
            if rt < time_before_next:
                print(f"Fin de la simulation à {datetime.fromtimestamp(env.now)} !")

            yield env.timeout(time_before_next)

        else:
            time_to_wait = datetime.timestamp(next_date) - env.now
            setup = True
            current_hour = datetime.fromtimestamp(env.now).hour
            rt = timestampnow + timedelta(days = nb_jours).total_seconds() - env.now
            if rt <= time_to_wait:
                print(f"Fin de la simulation à {datetime.fromtimestamp(env.now)} !")
            yield env.timeout(time_to_wait) 

In [52]:
timestampnow = datetime.timestamp(datetime(2020,7,6,6,0,0,0))
bidrequests =  namedtuple(
    "bidrequests", 
    (
        "identifiant", 
        "timestamp", 
        "timestamp_string",
        "nombre_impressions",
        "prix_impression",
        "prix_total"
    )
                    )
data = list()
env = simpy.Environment(initial_time=timestampnow)
proc = env.process(open_rtb_pacing_double_algo(env = env, P = 1, timestampnow = timestampnow, 
                                               nb_jours = 30, nb_hours_per_day = 14, 
                                               bidrequests = bidrequests, data = data))
env.run(until=timestampnow + timedelta(days = 30).total_seconds())

Fin de la simulation à 2020-08-04 20:07:34 !


In [53]:
sauvegarde(data, '06-07-2020_04-08-2020.csv')